In [34]:
import pandas as pd


In [35]:
data = pd.read_csv('all_fight_results.csv')
# Reverse dataframe
print(data)

       Weight class          Fighter 1 Result          Fighter 2  \
0         Flyweight  Alexandre Pantoja    def        Kai Asakura   
1      Welterweight  Shavkat Rakhmonov    def  Ian Machado Garry   
2       Heavyweight         Ciryl Gane    def   Alexander Volkov   
3     Featherweight     Bryce Mitchell    def        Kron Gracie   
4     Featherweight        Choi Doo-ho    def      Nate Landwehr   
...             ...                ...    ...                ...   
7960            NaN     Gerard Gordeau    def       Kevin Rosier   
7961            NaN       Ken Shamrock    def      Patrick Smith   
7962            NaN       Royce Gracie    def      Art Jimmerson   
7963            NaN       Kevin Rosier    def       Zane Frazier   
7964            NaN     Gerard Gordeau    def         Teila Tuli   

                                          Method  Round  Time    #  
0        Technical Submission (rear-naked choke)    2.0  2:05  715  
1     Decision (unanimous) (48–47, 48–47, 48–

In [36]:
df = data.iloc[::-1].reset_index(drop=True)
print(df)

       Weight class          Fighter 1 Result          Fighter 2  \
0               NaN     Gerard Gordeau    def         Teila Tuli   
1               NaN       Kevin Rosier    def       Zane Frazier   
2               NaN       Royce Gracie    def      Art Jimmerson   
3               NaN       Ken Shamrock    def      Patrick Smith   
4               NaN     Gerard Gordeau    def       Kevin Rosier   
...             ...                ...    ...                ...   
7960  Featherweight        Choi Doo-ho    def      Nate Landwehr   
7961  Featherweight     Bryce Mitchell    def        Kron Gracie   
7962    Heavyweight         Ciryl Gane    def   Alexander Volkov   
7963   Welterweight  Shavkat Rakhmonov    def  Ian Machado Garry   
7964      Flyweight  Alexandre Pantoja    def        Kai Asakura   

                                          Method  Round  Time    #  
0                                TKO (head kick)    NaN  0:26    1  
1                                  TKO (punch

In [37]:
print(pd.unique(df['Result']))

['def' 'vs']


In [38]:
import importlib
import elo_calculator

In [42]:

importlib.reload(elo_calculator)
from elo_calculator import EloCalculator, ExpDecayK

peak = {}
elo_cur = {}
elo_history = {}
record = {}
for index, row in df.iterrows():
    num = row['#']
    A = row['Fighter 1']
    B = row['Fighter 2']
    if(A not in elo_cur):
        elo_cur[A] = 1200 # League of Legends starts everyone at 1200 elo
        elo_history[A] = []
        record[A] = {"wins": 0, "losses": 0, "draws": 0}
    if(B not in elo_cur):
        elo_cur[B] = 1200
        elo_history[B] = []
        record[B] = {"wins": 0, "losses": 0, "draws": 0}
    A_old_elo = elo_cur[A]
    B_old_elo = elo_cur[B]
    A_fights = record[A]["wins"] + record[A]["losses"] + record[A]["draws"]
    B_fights = record[B]["wins"] + record[B]["losses"] + record[B]["draws"]
    # Update record
    if(row['Result'] == 'def'):
        winner = 1
        record[A]["wins"] += 1
        record[B]["losses"] += 1
    else:
        winner = 0.5
        record[A]["draws"] += 1
        record[B]["draws"] += 1
    A_record = f"{record[A]['wins']}-{record[A]['losses']}-{record[A]['draws']}"
    B_record = f"{record[B]['wins']}-{record[B]['losses']}-{record[B]['draws']}"
    Ka, Kb = ExpDecayK(A_fights, B_fights)
    A_new_elo, B_new_elo = EloCalculator(A_old_elo, B_old_elo, winner, Ka, Kb)
    # Update elo
    elo_cur[A] = A_new_elo
    elo_cur[B] = B_new_elo
    # Add to elo history
    elo_history[A].append({"Event #": num, "Elo Rating": A_new_elo, "Record": A_record})
    elo_history[B].append({"Event #": num, "Elo Rating": B_new_elo, "Record": B_record})
    # Update peak elo
    if(A not in peak or peak[A]['Elo'] <= A_new_elo):
        peak[A] = {"Event #": num, "Elo": A_new_elo}
    if(B not in peak or peak[B]['Elo'] <= B_new_elo):
        peak[B] = {"Event #": num, "Elo": B_new_elo}